# Para usar en otro codigo, será necesario cambiar las rutas de:
# Path - Aqui va la ubicación donde se encuentra el csv de la lista de Rucs
# service - En donde se encuentra ubicado el .exe de chromedriver
# archivo_salida - Donde irá guardado el archivo y el nombre que tendrá
# os.path.exists/ mkdirs - Ruta donde se guardará el archivo excel

In [ ]:
import easyocr
import pyautogui
import cv2
import os
import time 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import requests
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from datetime import datetime

In [ ]:
path=r'C:\Analytics\Lista_de_Ruc.csv'
df= pd.read_csv(path, sep=",")
df.head()

In [ ]:
Registros =df ['RUC'].unique()
Registros=Registros.astype(str)
print(Registros)

In [ ]:
# Configurar Selenium y Chrome
chrome_options = Options()
service = Service(r'C:\Analytics\chromedriver.exe')
driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
# Ruta del escritorio
escritorio = os.path.join(os.path.expanduser("~"), "Desktop", "Supercias_Datos")

# Verificar si la carpeta existe, si no, crearla
if not os.path.exists(escritorio):
    os.makedirs(escritorio, exist_ok=True)

# Ruta del archivo
captcha_path = os.path.join(escritorio, "captcha.png")
def get_imgORIG(image_src):
    # Descargar la imagen del CAPTCHA
    response = requests.get(image_src, stream=True, headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code == 200:
        with open(captcha_path, 'wb') as file:
            file.write(response.content)
    else:
        print("Error al descargar el CAPTCHA")
        return ""   
    
    captcha_path2 = cv2.imread(captcha_path)
  # Convertir la imagen a texto 
    reader = easyocr.Reader(["es"], gpu=False)
    result = reader.readtext(captcha_path2)
    
    # Unir los textos reconocidos en una sola cadena
    text = " ".join([res[1] for res in result])
    text = result[0][1].replace(" ", "")
    print("Texto detectado:", text)
    return text

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()
wait = WebDriverWait(driver, 10)

datos_extraidos= []

for registro in Registros:
    #hola
    driver.get ("https://appscvsgen.supercias.gob.ec/consultaCompanias/societario/busquedaCompanias.jsf")
    #print(pyautogui.position())
    
    #CLick en Radio Button
    #z,w=669,520
    #pyautogui.click(z,w, button='left')
    boton_ruc = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/form/div[3]/div/div[2]/table/tbody/tr[1]/td/table/tbody/tr/td[2]")))
    boton_ruc.click()

    #radio_button=wait.until(EC.visibility_of_element_located(By.XPATH, "/html/body/form/div[3]/div/div[2]/table/tbody/tr[1]/td/table/tbody/tr/td[2]/div/div[1]/input").click())
    #radio_button.click(button='left')
    time.sleep(2)
    
    #Ingreso RUC
    input_RUC=wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/form/div[3]/div/div[2]/table/tbody/tr[2]/td/span/input")))
    numero_de_registro=registro
    input_RUC.send_keys(numero_de_registro)
    time.sleep(1)
    input_RUC.send_keys(Keys.ENTER)
    
    #Rresolver captcha
    time.sleep(3)
    image_element = driver.find_element(By.ID, "frmBusquedaCompanias:captchaImage")
    image_src = image_element.get_attribute("src")
    print("url de imagen:",image_src)
    try:
        captcha= get_imgORIG(image_src)
        print("respuesta de funcion",captcha)
    except:
        pass
    print(registro)

    #input de captcha
    input_captcha =wait.until(EC.visibility_of_element_located((By.XPATH,"/html/body/form/div[3]/div/div[2]/table/tbody/tr[4]/td/span/div/div[2]/table/tbody/tr[1]/td/input")))
    input_captcha.send_keys(captcha)
    #time.sleep(0.5)
    input_captcha.send_keys(Keys.ENTER)
    url_actual=driver.current_url
    
    #--#
    #menulateral
    #print(pyautogui.position())        
    time.sleep(3) #click menu 
    #x,y=115,909
    #pyautogui.click(x,y, button='left')
    boton_cumplimiento = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[2]/form/div[1]/ul/li[28]/a")))
    boton_cumplimiento.click()
    time.sleep(3)# Esperar por si aparece otro CAPTCHA
    
#Verificar y resolver el CAPTCHA extra
    try:
        image_elementex = driver.find_element(By.ID, "frmCaptcha:captchaImage")
        image_src2 = image_elementex.get_attribute("src")
        print("URL CAPTCHA extra:", image_src2)
        try:
            captcha_extra_resuelto = get_imgORIG(image_src2)
            print("Respuesta CAPTCHA extra:", captcha_extra_resuelto)
        except Exception as e:
            print("Error al resolver el CAPTCHA extra:", e)
            captcha_extra_resuelto = ""
            
        url_antes = driver.current_url
        print("URL antes de enviar CAPTCHA:", url_antes)

        input_captchaEX = wait.until(EC.visibility_of_element_located((By.XPATH, "/html/body/div[11]/div[2]/form/div/div/table/tbody/tr[1]/td/input"))) 
        input_captchaEX.send_keys(captcha_extra_resuelto)
        boton_enviar_captcha = wait.until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[11]/div[2]/form/button")))
        boton_enviar_captcha.click()
        WebDriverWait(driver, 10).until(lambda d: d.current_url != url_antes)
        url_despues = driver.current_url
        print("URL después de enviar CAPTCHA:", url_despues)
    
        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/form/span[1]/div/div/table[1]/tbody/tr/td[2]/label")))
        #input_captchaEX.send_keys(Keys.ENTER)
    except Exception as e:
        print("No se encontró CAPTCHA extra:", e)
    
#Extracción de datos
    try:
        ruc=driver.find_element(By.XPATH,"/html/body/div[3]/div/form/span[1]/div/div/div/div/table[3]/tbody/tr[1]/td[2]/label").text.strip()
        representante = driver.find_element(By.XPATH, "/html/body/div[3]/div/form/span[1]/div/div/div/div/table[3]/tbody/tr[3]/td[2]/label").text.strip()
        cumplimiento= driver.find_element(By.XPATH, "/html/body/div[3]/div/form/span[1]/div/div/div/div/table[3]/tbody/tr[6]/td[2]/label").text.strip()
    except:
        ruc = representante = cumplimiento = "No disponible"

    fecha_consulta = datetime.now().strftime("%Y-%m-%d")
    hora_consulta = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
    # Crear un DataFrame solo con el nuevo dato
    nuevo_dato = pd.DataFrame([{
        "Fecha de Consulta": fecha_consulta,
        "Hora de Consulta": hora_consulta,
        "RUC": ruc,
        "Representante Legal": representante,
        "Cumplimiento": cumplimiento
    }])
    nuevo_dato["RUC"] = nuevo_dato["RUC"].astype(str)
    #archivo_salida = os.path.join(os.path.expanduser("~"), "Desktop", "cumplimiento_supercias.xlsx")
    
    archivo_salida = r"C:\Analytics\CUMPLIMIENTO\cumplimiento_supercias.xlsx"
    # Verificar si la carpeta existe, si no, crearla
    if not os.path.exists("C:\\Analytics\\CUMPLIMIENTO"):
        os.makedirs("C:\\Analytics\\CUMPLIMIENTO", exist_ok=True)
    if os.path.exists(archivo_salida):
        df_existente = pd.read_excel(archivo_salida, dtype={"RUC": str})
        df_resultado = pd.concat([df_existente, nuevo_dato]).drop_duplicates()
        with pd.ExcelWriter(archivo_salida, mode="w", engine="openpyxl") as writer:
            df_resultado.to_excel(writer, index=False)
    else:
        nuevo_dato.to_excel(archivo_salida, index=False)
        
    print("Datos guardados en:", archivo_salida)
    
        
driver.quit()